In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset


In [4]:
# Synthetic IoT commands and their intents
commands = [
    "turn on the living room fan",
     "turn on the bedroom light",
    "turn off the kitchen fan",
    "turn off the bedroom light",
    "set thermostat to 22 degrees",
    "increase humidity by 5 percent",
    "decrease brightness in the hallway",
    "check the bedroom temperature",
    "what is the current humidity level?"
]

intents = [
    "TURN_ON_DEVICE",
    "TURN_ON_DEVICE",
    "TURN_OFF_DEVICE",
    "TURN_OFF_DEVICE",
    "ADJUST_PARAMETER",
    "ADJUST_PARAMETER",
    "ADJUST_PARAMETER",
    "QUERY_STATUS",
    "QUERY_STATUS"
]

df = pd.DataFrame({'command': commands, 'intent': intents})
print(df)


                               command            intent
0          turn on the living room fan    TURN_ON_DEVICE
1            turn on the bedroom light    TURN_ON_DEVICE
2             turn off the kitchen fan   TURN_OFF_DEVICE
3           turn off the bedroom light   TURN_OFF_DEVICE
4         set thermostat to 22 degrees  ADJUST_PARAMETER
5       increase humidity by 5 percent  ADJUST_PARAMETER
6   decrease brightness in the hallway  ADJUST_PARAMETER
7        check the bedroom temperature      QUERY_STATUS
8  what is the current humidity level?      QUERY_STATUS


In [5]:
label_to_id = {label: i for i, label in enumerate(df['intent'].unique())}
id_to_label = {v: k for k, v in label_to_id.items()}
df['label'] = df['intent'].map(label_to_id)
print("Label mapping:", label_to_id)
print(df)


Label mapping: {'TURN_ON_DEVICE': 0, 'TURN_OFF_DEVICE': 1, 'ADJUST_PARAMETER': 2, 'QUERY_STATUS': 3}
                               command            intent  label
0          turn on the living room fan    TURN_ON_DEVICE      0
1            turn on the bedroom light    TURN_ON_DEVICE      0
2             turn off the kitchen fan   TURN_OFF_DEVICE      1
3           turn off the bedroom light   TURN_OFF_DEVICE      1
4         set thermostat to 22 degrees  ADJUST_PARAMETER      2
5       increase humidity by 5 percent  ADJUST_PARAMETER      2
6   decrease brightness in the hallway  ADJUST_PARAMETER      2
7        check the bedroom temperature      QUERY_STATUS      3
8  what is the current humidity level?      QUERY_STATUS      3


In [6]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print("Training data:")
print(train_df)
print("Test data:")
print(test_df)


Training data:
                               command            intent  label
5       increase humidity by 5 percent  ADJUST_PARAMETER      2
0          turn on the living room fan    TURN_ON_DEVICE      0
8  what is the current humidity level?      QUERY_STATUS      3
2             turn off the kitchen fan   TURN_OFF_DEVICE      1
4         set thermostat to 22 degrees  ADJUST_PARAMETER      2
3           turn off the bedroom light   TURN_OFF_DEVICE      1
6   decrease brightness in the hallway  ADJUST_PARAMETER      2
Test data:
                         command          intent  label
7  check the bedroom temperature    QUERY_STATUS      3
1      turn on the bedroom light  TURN_ON_DEVICE      0


In [7]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Keep only necessary columns for the model
train_dataset = train_dataset.remove_columns(["intent", "__index_level_0__"])
test_dataset = test_dataset.remove_columns(["intent", "__index_level_0__"])


In [8]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(example):
    return tokenizer(example["command"], truncation=True, padding='max_length', max_length=32)

train_encoded = train_dataset.map(tokenize_function, batched=True)
test_encoded = test_dataset.map(tokenize_function, batched=True)

train_encoded = train_encoded.remove_columns(["command"])
test_encoded = test_encoded.remove_columns(["command"])

train_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=len(label_to_id))


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    output_dir="results",
    evaluation_strategy="steps",
    num_train_epochs=20,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=1,
    save_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=test_encoded
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer.train()
trainer.evaluate()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kapooramita. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
10,1.146800,1.650992
20,0.605900,1.992197
30,0.314000,1.704657
40,0.230800,1.639256


{'eval_loss': 1.6392557621002197,
 'eval_runtime': 0.2583,
 'eval_samples_per_second': 7.742,
 'eval_steps_per_second': 3.871,
 'epoch': 20.0}

In [14]:
test_command = "turn on the fan"
inputs = tokenizer(test_command, return_tensors="pt", truncation=True, padding='max_length', max_length=32)
# Remove token_type_ids from inputs
inputs = {key: value for key, value in inputs.items() if key != "token_type_ids"}
with torch.no_grad():
    outputs = model(**inputs)
    predicted_label_id = outputs.logits.argmax(dim=-1).item()

predicted_intent = id_to_label[predicted_label_id]
print(f"Command: {test_command}")
print(f"Predicted Intent: {predicted_intent}")


Command: turn on the fan
Predicted Intent: TURN_ON_DEVICE
